In [1]:
import warnings

warnings.filterwarnings("ignore")
import os

if "jbook" in os.getcwd():
    os.chdir(os.path.abspath(os.path.join("../..")))
FORCE = True

# Text Quality Detection
Noise removal is a standard part of many NLP pipelines, and for good reason: machine learning models often struggle with inconsistencies and irrelevant artifacts in text data, which can degrade performance. Traditional models, in particular, were highly sensitive to noise, requiring rigorous preprocessing to function effectively.

However, transformer models, such as BERT, which was trained on a corpus of approximately **3.3 billion words** from sources like BooksCorpus and English Wikipedia, have demonstrated remarkable robustness to linguistic noise. Their ability to handle variations in text, such as abbreviations, emojis, slang, internet jargon, grammatical errors, informal word forms, and misspellings, stems from a combination of subword tokenization techniques and the innovative self-attention mechanism.

1. **Misspellings**: Subword tokenization techniques, like Byte Pair Encoding (BPE) and SentencePiece, break misspelled words into recognizable subword components. This allows models to leverage existing subword embeddings and infer the intended meaning, even when spelling deviations occur. The model still maintains high semantic accuracy because it can assemble meaning from familiar patterns rather than needing a perfect match.

2. **Slang and Informal Language**: Transformers are trained on diverse, real-world text that includes slang and informal expressions, making them adept at understanding and processing these variations. Subword tokenization decomposes these unconventional words into smaller units that the model has encountered in other contexts, enabling generalization. Additionally, transformers’ vast training data captures the distribution and use of slang, embedding these linguistic nuances effectively.

3. **Emojis and Special Characters**: Subword tokenization treats emojis and special symbols as unique tokens, preserving their semantic value. The self-attention mechanism allows the model to integrate these elements contextually, understanding their contribution to sentiment or meaning within the text. By attending to the relationships between emojis and surrounding words, the model can interpret and generate text that accurately reflects emotional tone or emphasis.

4. **Abbreviations and Internet Jargon**: Abbreviations and internet-specific language are broken down into meaningful subword segments, allowing transformers to recognize patterns and relate them to standard language forms. The self-attention mechanism plays a crucial role here by dynamically assigning importance to different parts of the input sequence, enabling the model to understand the intended message despite the use of abbreviations.

5. **Grammatical Errors and Informal Word Forms**: The self-attention mechanism is a fundamental innovation in transformer models. It enables the model to establish contextual relationships between words regardless of their order or grammatical correctness. By weighing the relevance of each word in relation to others, the model captures the overarching meaning even in the presence of syntax errors or informal language structures. This flexibility makes transformers robust to variations that would otherwise disrupt traditional models.

Moreover, studies have shown that some types of "useful" noise, such as informal language and emojis, can enhance model performance and generalizability, as they better simulate real-world text scenarios {cite}`languageandmultimodalailamalabimperialcollegelondonukBetterUnderstandingNoise2021`. By preserving or even embracing this *useful* noise, models become more adaptable and effective in practical applications, demonstrating the nuanced trade-offs in handling linguistic noise.

Therefore, we take a nuanced, task-specific approach to noise handling, isolating and removing only *harmful* noise. We define harmful noise as artifacts that do not carry meaning or distort the intended meaning of the text. To ensure high data quality, we assess and flag observations based on several dimensions:

- **Personally Identifiable Information (PII)**: This includes URLs, emails, and phone numbers, which can compromise privacy.
- **Language Noise**: We flag non-English text, which can hinder language-specific models from accurately interpreting content.
- **Accuracy Noise**: Artifacts such as control characters, HTML tags, excessive whitespace, elongation of characters, non-ASCII characters, and certain special characters that disrupt text consistency are flagged and managed.
- **Validity**: We identify review length and perplexity outliers that may indicate irregular, fake or otherwise invalid content. For `review_length` and `perplexity` we take a conservative approach. An outlier is defined as a value beyond $3\times\text{IQR}$

By addressing these dimensions, we optimize data quality in a way that enhances model performance without sacrificing the rich, real-world variability that some noise can provide.

## Import Libraries

In [2]:
from discover.container import DiscoverContainer
from discover.infra.config.flow import FlowConfigReader
from discover.core.flow import StageDef
from discover.flow.data_prep.dqm.stage import DataQualityDetectionStage
from discover.core.flow import PhaseDef, StageDef

## Dependency Container

In [3]:
container = DiscoverContainer()
container.init_resources()
container.wire(
    modules=[
        "discover.flow.data_prep.base.stage",
    ],
)

## Text Quality Detection (DQA) Pipeline
This text quality detection pipeline flags observations containing *harmful* noise artifacts for downstream transformation, replacement, or removal. 

In [4]:
# Obtain the configuration
reader = FlowConfigReader()
stage_config = reader.get_stage_config(
    phase=PhaseDef.DATAPREP, stage=StageDef.DQD
)
# Build and run the stage
stage = DataQualityDetectionStage.build(stage_config=stage_config, force=FORCE)
asset_id = stage.run()

[11/16/2024 12:49:23 PM] [INFO] [discover.infra.persistence.repo.dataset.DatasetRepo] [_remove_dataset_file_by_filepath] : Removed dataset file at workspace/dev/dataset/01_dataprep/appvocai_discover-01_dataprep-04_tqd-review-dataset.parquet from repository.
[11/16/2024 12:49:23 PM] [INFO] [discover.infra.persistence.repo.dataset.DatasetRepo] [remove] : Removed dataset dataset-dev-dataprep-tqd-review from the repository.




# ============================================================================== #
#                          Text Quality Detection Stage                          #
# ============================================================================== #



your 131072x1 screen size is bogus. expect trouble


:: loading settings :: url = jar:file:/home/john/miniconda3/envs/appvocai/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/john/.ivy2/cache
The jars for the packages stored in: /home/john/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-71cba5f1-a0d3-4261-bb9c-2889adfbaa3d;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	f



                             DetectOrRepairURLTask                              
                             ---------------------                              
                          Start Datetime | Sat, 16 Nov 2024 12:49:49
                       Complete Datetime | Sat, 16 Nov 2024 12:49:49
                                 Runtime | 0.2 seconds


                         DetectOrRepairEmailAddressTask                         
                         ------------------------------                         
                          Start Datetime | Sat, 16 Nov 2024 12:49:49
                       Complete Datetime | Sat, 16 Nov 2024 12:49:50
                                 Runtime | 0.08 seconds


                         DetectOrRepairPhoneNumberTask                          
                         -----------------------------                          
                          Start Datetime | Sat, 16 Nov 2024 12:49:50
                       Complete Datetime | Sat, 16 N

                       Complete Datetime | Sat, 16 Nov 2024 12:49:54
                                 Runtime | 3.52 seconds


                           DetectOrRepairOutliersTask                           
                           --------------------------                           
                          Start Datetime | Sat, 16 Nov 2024 12:49:54
                       Complete Datetime | Sat, 16 Nov 2024 12:49:55
                                 Runtime | 0.7 seconds




                          Text Quality Detection Stage                          
                           Stage Started | Sat, 16 Nov 2024 12:49:23
                         Stage Completed | Sat, 16 Nov 2024 12:55:34
                           Stage Runtime | 6.0 minutes and 10.56 seconds


# ============================================================================ #



With **Text Quality Detection** step complete, our dataset is now enriched with quality signals for a meaningful **Data Quality Analysis (DQA)**, where we’ll examine the broader impact of these artifacts on overall data quality. In the next section, we will analyze review text through the lens of syntactic complexity, lexical diversity, and coherence - linguistic characteristics associated with high quality reviews.